In [1]:
import math
import numpy as np
import sys,os

In [2]:
poly_dir = "Polynomials_Saved_Datasets"
if not os.path.exists(poly_dir):
    os.makedirs(poly_dir)

In [3]:
def isprime(n) :
    if (n <= 1):
        return False
    if (n <= 3):
        return True
 
    # This is checked so that we can skip
    # middle five numbers in below loop
    if (n % 2 == 0 or n % 3 == 0):
        return False
 
    i = 5
    while(i * i <= n):
        if (n % i) == 0 or n % (i + 2) == 0:
            return False
        i = i + 6
 
    return True

# Inverse of a Distribution for Primes of the Form a^n + D*b^n, D>0

In [4]:
def inverse_distribution_primes_a_b_nth(D, n):
    delta_x=0.0001
    integral = 0
    i_lst = []
    counter = 0
    limit = 0.1
    i_ = 0
    while counter <10:
        for j in range(i_+1,10000):
            integral += delta_x*(1/ (1+D * (j*delta_x)**n)**(2/n)) #(nrt(1 + D * (j*delta_x) **n,n/2)))
            #print (integral, "\t", limit , i_lst)
            if integral>limit:
                i_ = j
                i_lst.append(j)
                break
        limit = limit + 0.1
        counter+=1
    return i_lst

# Distribution of prime numbers of the form a^n + D*b^n,D>0

In [5]:
def list_creator(n):
    List = []
    for i in range(n):
        List.append(0)
    return List

In [6]:
def distribution_primes_a_b(N, D, n, LIST):
    len_List = len(LIST)
    counter_list = list_creator(len_List)
    for a in range(1,N):
        for b in range(1,a):
            q = a**n + D*b**n
            M = (N**n)/(D+1)
            if q<M and isprime(q):
                for i in range(len(LIST)):
                    if i==0 and b/a <= LIST[0]/10000:
                        counter_list[0]+=1 
                    elif i!=0 and b/a <= LIST[i]/10000 and b/a > LIST[i-1]/10000:
                        counter_list[i]+=1 
    return counter_list
    

# Distribution of prime numbers of the form a^n + D*b^n, D>0 in Modulo 2

In [14]:
def distribution_primes_a_b_mod2(N, D, n, LIST):
    len_List = len(LIST)
    odd_odd = list_creator(len_List)#an empty when a is odd and b is odd4
    odd_even = list_creator(len_List)#an empty when a is odd and b is even
    even_odd = list_creator(len_List)#an empty when a is even and b is odd
    for a in range(1,N):
        for b in range(1,a):
            if a%2 == 1 and b%2 == 1:
                q = a**n + D*b**n
                M = (N**n)/(D+1)
                if q<M and isprime(q):
                    for i in range(len(LIST)):
                        if i==0 and b/a <= LIST[0]/10000:
                            odd_odd[0]+=1 
                        elif i!=0 and b/a <= LIST[i]/10000 and b/a > LIST[i-1]/10000:
                            odd_odd[i]+=1 
            elif a%2 == 1 and b%2 == 0:
                q = a**n + D*b**n
                M = (N**n)/(D+1)
                if q<M and isprime(q):
                    for i in range(len(LIST)):
                        if i==0 and b/a <= LIST[0]/10000:
                            odd_even[0]+=1 
                        elif i!=0 and b/a <= LIST[i]/10000 and b/a > LIST[i-1]/10000:
                            odd_even[i]+=1
            elif D%2 == 1 and a%2 == 0 and b%2 == 1:
                q = a**n + D*b**n
                M = (N**n)/(D+1)
                if q<M and isprime(q):
                    for i in range(len(LIST)):
                        if i==0 and b/a <= LIST[0]/10000:
                            even_odd[0]+=1 
                        elif i!=0 and b/a <= LIST[i]/10000 and b/a > LIST[i-1]/10000:
                            even_odd[i]+=1 
    if D%2 == 1:
        return even_odd + odd_even
    else:
        return odd_odd + odd_even

In [10]:
def deviation(dist_list):
    return max(dist_list)/min(dist_list)

In [11]:
def testing_d_pos(rangeN, incre, D, n):
    print(f"Primes of the form a^{n} + {D}b^{n}")
    print()
    print("Calculating the inverse distribution..")
    inverse_list = inverse_distribution_primes_a_b_nth(D, n)
    
    print ("Calculating the distribution of primes....")
    test_arr = np.zeros((rangeN//incre, len(inverse_list)),  dtype=np.int32)
    test_arrMod2 = np.zeros((rangeN//incre, len(inverse_list)),  dtype=np.int32)
    i = 0
    for N in range(incre,rangeN+incre, incre):
        test_arr[i] = distribution_primes_a_b(N, D, n, inverse_list)
        #print (f"N = {N}  ", str(distribution_primes_a_b(N, D, n, inverse_list)), deviation(distribution_primes_a_b(N, D, n, inverse_list)))
        i+=1
    
    print ("Calculating the distribution of primes in mod2....")
    columns = len(distribution_primes_a_b_mod2(100, D, n, inverse_list))
    test_arrMod2 = np.zeros((rangeN//incre, columns),  dtype=np.int32)
    j = 0
    for Nn in range(incre,rangeN+incre, incre):
        test_arrMod2[j] = distribution_primes_a_b_mod2(Nn, D, n, inverse_list)
        #print (f"N = {N}  ", str(distribution_primes_a_b(N, D, n, inverse_list)), deviation(distribution_primes_a_b(N, D, n, inverse_list)))
        j+=1
    print("Saving my arrays in npz file.....")
    np.savez("Polynomials_Saved_Datasets/"+str(D)+ "_power_"+ str(n) +"_"+ str(incre)+"_to_"+str(rangeN), inverse = np.array(inverse_list) ,distri = test_arr, distriMod2 = test_arrMod2)
    print("Done")

# Inverse of a Distribution for Primes of the Form a^n + D*b^n, D<0

In [20]:
def neg_inverse_distribution_primes_a_b_nth(D, n):
    delta_x=0.0001
    integral = 0
    i_lst = []
    counter = 0
    limit = 0.1
    i_ = 0
    while counter <6:
        for j in range(i_+1,10000):
            integral += 4*delta_x*(1/ (1+D * (j*delta_x)**n)**(2/n)) #(nrt(1 + D * (j*delta_x) **n,n/2)))
            #print (integral, "\t", limit , i_lst)
            if integral>limit:
                i_ = j
                i_lst.append(j)
                break
        limit = limit + 0.1
        counter+=1
    return i_lst

# Distribution of prime numbers of the form a^n + D*b^n, D<0

In [21]:
def neg_distribution_primes_a_b(N, D, n, LIST):
    len_List = len(LIST)
    counter_list = list_creator(len_List)
    for a in range(1,N):
        l = math.floor(a/((-D)**(1/n)))
        for b in range(1,l):
            q = a**n + D*b**n
            M = (N**n) - ((N**n)/(2**n*(-D)))
            if q<M and isprime(q):
                for i in range(len(LIST)):
                    if i==0 and b/a <= LIST[0]/10000:
                        counter_list[0]+=1 
                    elif i!=0 and b/a <= LIST[i]/10000 and b/a > LIST[i-1]/10000:
                        counter_list[i]+=1 
    return counter_list

# Distribution of prime numbers of the form a^n + D*b^n, D<0 Modulo 2

In [22]:
def neg_distribution_primes_a_b_mod2(N, D, n, LIST):
    len_List = len(LIST)
    odd_odd = list_creator(len_List)#an empty when a is odd and b is odd4
    odd_even = list_creator(len_List)#an empty when a is odd and b is even
    even_odd = list_creator(len_List)#an empty when a is even and b is odd
    for a in range(1,N):
        l = math.floor(a/((-D)**(1/n)))
        for b in range(1,l):
            if a%2 == 1 and b%2 == 1:
                q = a**n + D*b**n
                M = (N**n) - ((N**n)/(2**n*(-D)))
                if q<M and isprime(q):
                    for i in range(len(LIST)):
                        if i==0 and b/a <= LIST[0]/10000:
                            odd_odd[0]+=1 
                        elif i!=0 and b/a <= LIST[i]/10000 and b/a > LIST[i-1]/10000:
                            odd_odd[i]+=1 
            elif a%2 == 1 and b%2 == 0:
                q = a**n + D*b**n
                M = (N**n) - ((N**n)/(2**n*(-D)))
                if q<M and isprime(q):
                    for i in range(len(LIST)):
                        if i==0 and b/a <= LIST[0]/10000:
                            odd_even[0]+=1 
                        elif i!=0 and b/a <= LIST[i]/10000 and b/a > LIST[i-1]/10000:
                            odd_even[i]+=1
            elif D%2 == 1 and a%2 == 0 and b%2 == 1:
                q = a**n + D*b**n
                M = (N**n) - ((N**n)/(2**n*(-D)))
                if q<M and isprime(q):
                    for i in range(len(LIST)):
                        if i==0 and b/a <= LIST[0]/10000:
                            even_odd[0]+=1 
                        elif i!=0 and b/a <= LIST[i]/10000 and b/a > LIST[i-1]/10000:
                            even_odd[i]+=1 
    if D%2 == 1:
        return even_odd + odd_even
    else:
        return odd_odd + odd_even

In [27]:
def testing_d_neg(rangeN, incre, D, n):
    print(f"Primes of the form a^{n} + {D}b^{n}")
    print()
    print("Calculating the negative inverse distribution..")
    neg_inverse_list = neg_inverse_distribution_primes_a_b_nth(D, n)
    
    print ("Calculating the distribution of primes D<0....")
    test_arr = np.zeros((rangeN//incre, len(neg_inverse_list)),  dtype=np.int32)
    i = 0
    for N in range(incre,rangeN+incre, incre):
        test_arr[i] = neg_distribution_primes_a_b(N, D, n, neg_inverse_list)
        #print (f"N = {N}  ", str(distribution_primes_a_b(N, D, n, inverse_list)), deviation(distribution_primes_a_b(N, D, n, inverse_list)))
        i+=1
    
    print ("Calculating the distribution of primes in mod2 D<0....")
    columns = len(distribution_primes_a_b_mod2(100, D, n, neg_inverse_list))
    test_arrMod2 = np.zeros((rangeN//incre, columns),  dtype=np.int32)
    j = 0
    for Nn in range(incre,rangeN+incre, incre):
        test_arrMod2[j] = neg_distribution_primes_a_b_mod2(Nn, D, n, neg_inverse_list)
        #print (f"N = {N}  ", str(distribution_primes_a_b(N, D, n, inverse_list)), deviation(distribution_primes_a_b(N, D, n, inverse_list)))
        j+=1
    print("Saving my arrays in npz file.....")
    np.savez("Polynomials_Saved_Datasets/"+"neg_"+str(abs(D))+ "_power_"+ str(n) +"_"+ str(incre)+"_to_"+str(rangeN), inverse = np.array(neg_inverse_list) ,distri = test_arr, distriMod2 = test_arrMod2)
    print("Done")

# Testing: Inverse Distruntion for primes a^2 + b^2

In [12]:
testing_d_pos(4000, 500, 1, 2)


Primes of the form a^2 + 1b^2

Calculating the inverse distribution..
Calculating the distribution of primes....
Calculating the distribution of primes in mod2....
Saving my arrays in npz file.....
Done


## Testing: Inverse Distruntion for primes a^2 + 2b^2

In [16]:
testing_d_pos(4000, 500, 2, 2)

Primes of the form a^2 + 2b^2

Calculating the inverse distribution..
Calculating the distribution of primes....
Calculating the distribution of primes in mod2....
Saving my arrays in npz file.....
Done


# Testing: Inverse Distruntion for primes a^2 + 3b^2

In [17]:
testing_d_pos(4000, 500, 3, 2)

Primes of the form a^2 + 3b^2

Calculating the inverse distribution..
Calculating the distribution of primes....
Calculating the distribution of primes in mod2....
Saving my arrays in npz file.....
Done


# Testing: Inverse Distruntion for primes a^2 + 5b^2

In [18]:
testing_d_pos(4000, 500, 5, 2)

Primes of the form a^2 + 5b^2

Calculating the inverse distribution..
Calculating the distribution of primes....
Calculating the distribution of primes in mod2....
Saving my arrays in npz file.....
Done


# Testing: Inverse Distruntion for primes a^2 + 6b^2

In [19]:
testing_d_pos(4000, 500, 6, 2)

Primes of the form a^2 + 6b^2

Calculating the inverse distribution..
Calculating the distribution of primes....
Calculating the distribution of primes in mod2....
Saving my arrays in npz file.....
Done


# Testing: Inverse Distribution for primes a^2 - 3b^2

In [28]:
testing_d_neg(4000, 500, -3, 2)

Primes of the form a^2 + -3b^2

Calculating the negative inverse distribution..
Calculating the distribution of primes D<0....
Calculating the distribution of primes in mod2 D<0....
Saving my arrays in npz file.....
Done


# Testing: Inverse Distruntion for primes a^2-2b^2

In [37]:
testing_d_neg(4000, 500, -2, 2)

Primes of the form a^2 + -2b^2

Calculating the negative inverse distribution..
Calculating the distribution of primes D<0....
Calculating the distribution of primes in mod2 D<0....
Saving my arrays in npz file.....
Done


In [54]:
n_two_second = neg_inverse_distribution_primes_a_b_nth(-2,2)
n_two_second

[250, 500, 748, 994, 1238, 1478]

In [90]:
neg_testing(4000, 500, -2, 2, n_two_second) #rangeN, D, n, inverse_list

Distribution of primes of the form a^2 + -2b^2


{'N = 500  [378, 384, 399, 408, 380, 379]': 1.0793650793650793,
 'N = 1000  [1356, 1386, 1371, 1408, 1433, 1356]': 1.0567846607669618,
 'N = 1500  [2847, 2992, 2888, 2961, 2950, 2923]': 1.050930804355462,
 'N = 2000  [4875, 5153, 4987, 5000, 5049, 4973]': 1.0570256410256411,
 'N = 2500  [7445, 7734, 7592, 7609, 7616, 7530]': 1.0388179986568167,
 'N = 3000  [10547, 10789, 10664, 10639, 10598, 10673]': 1.0229449132454727,
 'N = 3500  [14080, 14372, 14216, 14190, 14193, 14246]': 1.0207386363636364,
 'N = 4000  [18099, 18396, 18291, 18161, 18217, 18230]': 1.0164097463948285}

In [ ]:
'N = 500  [762, 807, 759, 762]': 1.0632411067193677,
 'N = 1000  [2742, 2779, 2789, 2788]': 1.0171407731582787,
 'N = 1500  [5839, 5849, 5873, 5947]': 1.0184963178626476,
 'N = 2000  [10028, 9987, 10022, 10107]': 1.0120156203063984,
 'N = 2500  [15179, 15201, 15146, 15298]': 1.010035652977684,
 'N = 3000  [21336, 21303, 21271, 21470]': 1.009355460486108,
 'N = 3500  [28452, 28406, 28439, 28509]': 1.0036259945082024,
 'N = 4000  [36495, 36452, 36447, 36570]': 1.0033747633550087}

In [55]:
neg_mod2_testing(4000,500,-2,2,n_two_second)

Distribution of primes of the form a^2 + -2b^2
N = 500   [196, 194, 198, 214, 193, 185, 182, 190, 201, 194, 187, 194] 1.1758241758241759
N = 1000   [701, 695, 679, 714, 718, 681, 655, 691, 692, 694, 715, 675] 1.0961832061068701
N = 1500   [1450, 1517, 1445, 1469, 1492, 1465, 1397, 1475, 1443, 1492, 1458, 1458] 1.085898353614889
N = 2000   [2467, 2569, 2506, 2478, 2542, 2516, 2408, 2584, 2481, 2522, 2507, 2457] 1.0730897009966778
N = 2500   [3806, 3839, 3784, 3769, 3829, 3816, 3639, 3895, 3808, 3840, 3787, 3714] 1.0703489969771915
N = 3000   [5362, 5387, 5340, 5235, 5304, 5372, 5185, 5402, 5324, 5404, 5294, 5301] 1.0422372227579557
N = 3500   [7131, 7207, 7115, 6970, 7109, 7165, 6949, 7165, 7101, 7220, 7084, 7081] 1.0389984170384228
N = 4000   [9160, 9199, 9130, 8978, 9102, 9129, 8939, 9197, 9161, 9183, 9115, 9101] 1.0290860275198568


# Testing: Inverse Distruntion for primes a^2-5b^2

In [38]:
testing_d_neg(4000, 500, -5, 2)

Primes of the form a^2 + -5b^2

Calculating the negative inverse distribution..
Calculating the distribution of primes D<0....
Calculating the distribution of primes in mod2 D<0....
Saving my arrays in npz file.....
Done


In [56]:
n_five_second = neg_inverse_distribution_primes_a_b_nth(-5,2)
n_five_second

[250, 498, 744, 984, 1219, 1447]

In [85]:
neg_testing(4000, 500, -5, 2, n_five_second) #rangeN, D, n, inverse_list

Distribution of primes of the form a^2 + -5b^2


{'N = 500  [377, 411, 401, 411, 401, 406]': 1.090185676392573,
 'N = 1000  [1392, 1453, 1471, 1433, 1449, 1389]': 1.0590352771778258,
 'N = 1500  [3025, 3036, 3094, 3049, 3048, 2911]': 1.0628649948471316,
 'N = 2000  [5193, 5178, 5270, 5197, 5168, 4991]': 1.0559006211180124,
 'N = 2500  [7941, 7831, 7974, 7860, 7848, 7559]': 1.0549014419896812,
 'N = 3000  [11147, 11048, 11144, 11061, 11056, 10664]': 1.0452925731432858,
 'N = 3500  [14805, 14730, 14874, 14786, 14699, 14283]': 1.0413778617937408,
 'N = 4000  [18979, 18889, 19032, 18958, 18840, 18311]': 1.0393752389274207}

In [57]:
neg_mod2_testing(4000,500,-5,2,n_five_second)

Distribution of primes of the form a^2 + -5b^2
N = 500   [203, 208, 209, 210, 201, 189, 174, 203, 192, 201, 200, 217] 1.2471264367816093
N = 1000   [726, 727, 762, 729, 721, 656, 666, 726, 709, 704, 728, 733] 1.1615853658536586
N = 1500   [1557, 1504, 1551, 1538, 1525, 1436, 1468, 1532, 1543, 1511, 1523, 1475] 1.0842618384401115
N = 2000   [2652, 2563, 2623, 2620, 2601, 2475, 2541, 2615, 2647, 2577, 2567, 2516] 1.0715151515151515
N = 2500   [4019, 3885, 4001, 3929, 3969, 3751, 3922, 3946, 3973, 3931, 3879, 3808] 1.071447613969608
N = 3000   [5634, 5461, 5618, 5530, 5607, 5290, 5513, 5587, 5526, 5531, 5449, 5374] 1.0650283553875237
N = 3500   [7466, 7338, 7469, 7393, 7445, 7092, 7339, 7392, 7405, 7393, 7254, 7191] 1.0531584884376763
N = 4000   [9545, 9421, 9509, 9447, 9546, 9119, 9434, 9468, 9523, 9511, 9294, 9192] 1.0468253097927405


# Testing: Inverse Distruntion for primes a^3-2b^3

In [41]:
testing_d_neg(3000, 500, -2, 3)

Primes of the form a^3 + -2b^3

Calculating the negative inverse distribution..
Calculating the distribution of primes D<0....
Calculating the distribution of primes in mod2 D<0....
Saving my arrays in npz file.....
Done


In [12]:
n_two_third = neg_inverse_distribution_primes_a_b_nth(-2,3)
n_two_third

[250, 500, 750, 1000, 1250, 1499]

In [97]:
neg_testing(2000, 500, -2, 3, n_two_third) #rangeN, D, n, inverse_list

Distribution of primes of the form a^3 + -2b^3
N = 500   [213, 210, 238, 220, 212, 252] 1.2
N = 1000   [766, 803, 819, 808, 784, 834] 1.0887728459530026
N = 1500   [1631, 1679, 1673, 1658, 1667, 1709] 1.0478234212139792
N = 2000   [2853, 2900, 2863, 2866, 2906, 2873] 1.018576936558009


In [25]:
neg_mod2_testing(4000,500,-2,3,n_two_third)

Distribution of primes of the form a^3 + -2b^3
N = 500   [115, 98, 127, 105, 103, 128, 98, 112, 111, 115, 109, 124] 1.3061224489795917
N = 1000   [396, 396, 419, 390, 391, 429, 370, 407, 400, 418, 393, 405] 1.1594594594594594
N = 1500   [833, 829, 856, 803, 839, 844, 798, 850, 817, 855, 828, 865] 1.0839598997493733
N = 2000   [1456, 1437, 1437, 1413, 1459, 1448, 1397, 1463, 1426, 1453, 1447, 1425] 1.047244094488189
N = 2500   [2234, 2158, 2175, 2167, 2158, 2167, 2149, 2193, 2175, 2182, 2158, 2201] 1.0395532805956258
N = 3000   [3107, 3006, 3053, 3064, 3063, 3067, 2986, 3056, 3053, 3024, 3061, 3086] 1.0405224380442062


KeyboardInterrupt: 

# Testing: Inverse Distruntion for primes a^3-5b^3

In [ ]:
testing_d_neg(3000, 500, -5, 3)

Primes of the form a^3 + -5b^3

Calculating the negative inverse distribution..
Calculating the distribution of primes D<0....


In [28]:
n_five_third = neg_inverse_distribution_primes_a_b_nth(-5,3)
n_five_third

[250, 500, 750, 1000, 1248, 1496]

In [29]:
neg_testing(2000, 500, -5, 3, n_five_third) #rangeN, D, n, inverse_list

Distribution of primes of the form a^3 + -5b^3
N = 500   [185, 218, 196, 210, 192, 211] 1.1783783783783783
N = 1000   [691, 749, 717, 733, 721, 762] 1.1027496382054993
N = 1500   [1506, 1565, 1523, 1503, 1552, 1578] 1.0499001996007984
N = 2000   [2593, 2654, 2586, 2600, 2611, 2680] 1.0363495746326372


In [30]:
neg_mod2_testing(2000,500,-5,3,n_five_third)

Distribution of primes of the form a^3 + -5b^3
N = 500   [99, 109, 94, 104, 102, 100, 86, 109, 102, 106, 90, 111] 1.2906976744186047
N = 1000   [354, 367, 352, 362, 365, 362, 337, 382, 365, 371, 356, 400] 1.1869436201780414
N = 1500   [756, 781, 767, 739, 796, 746, 750, 784, 756, 764, 756, 832] 1.1258457374830853
N = 2000   [1304, 1333, 1302, 1294, 1328, 1288, 1289, 1321, 1284, 1306, 1283, 1392] 1.0849571317225253


# Testing: Inverse Distruntion for primes a^3-6b^3

In [ ]:
testing_d_neg(3000, 500, -6, 3)

In [31]:
n_six_third = neg_inverse_distribution_primes_a_b_nth(-6,3)
n_six_third

[250, 500, 750, 999, 1248, 1495]

In [32]:
neg_testing(2000, 500, -6, 3, n_six_third) #rangeN, D, n, inverse_list

Distribution of primes of the form a^3 + -6b^3
N = 500   [139, 162, 159, 164, 160, 155] 1.1798561151079137
N = 1000   [521, 565, 584, 562, 559, 547] 1.1209213051823417
N = 1500   [1115, 1195, 1229, 1169, 1182, 1147] 1.1022421524663677
N = 2000   [1939, 2008, 2078, 1988, 2026, 2000] 1.071686436307375


In [33]:
neg_mod2_testing(2000,500,-6,3,n_six_third)

Distribution of primes of the form a^3 + -6b^3
N = 500   [79, 73, 71, 81, 84, 73, 60, 89, 88, 83, 76, 82] 1.4833333333333334
N = 1000   [264, 285, 280, 287, 277, 273, 257, 280, 304, 275, 282, 274] 1.1828793774319066
N = 1500   [562, 611, 597, 599, 585, 568, 553, 584, 632, 570, 597, 579] 1.1428571428571428
N = 2000   [974, 1005, 1014, 1023, 1008, 1002, 965, 1003, 1064, 965, 1018, 998] 1.1025906735751296


# Testing: Inverse Distruntion for primes a^4-2b^4

In [ ]:
testing_d_neg(2000, 500, -2, 4)

In [17]:
neg_testing(1000, 500, -2, 4, n_two_fourth) #rangeN, D, n, inverse_list

Distribution of primes of the form a^4 + -2b^4
N = 500   [263, 274, 251, 230, 255, 252] 1.191304347826087
N = 1000   [902, 910, 921, 884, 885, 937] 1.0599547511312217


KeyboardInterrupt: 

In [18]:
neg_mod2_testing(2000,500,-2,4,n_two_fourth)

Distribution of primes of the form a^4 + -2b^4
N = 500   [133, 126, 138, 117, 120, 126, 130, 148, 113, 113, 135, 126] 1.3097345132743363


KeyboardInterrupt: 

# Testing: Inverse Distruntion for primes a^4-3b^4

In [ ]:
n_three_fourth = neg_inverse_distribution_primes_a_b_nth(-3,4)
n_three_fourth

In [ ]:
neg_testing(2000, 500, -3, 4, n_three_fourth) #rangeN, D, n, inverse_list

In [ ]:
neg_mod2_testing(2000,500,-3,4,n_three_fourth)

# Testing: Inverse Distruntion for primes a^4-5b^4

In [ ]:
n_five_fourth = neg_inverse_distribution_primes_a_b_nth(-5,4)
n_five_fourth

In [ ]:
neg_testing(2000, 500, -5, 4, n_five_fourth) #rangeN, D, n, inverse_list

In [ ]:
neg_mod2_testing(2000,500,-5,4,n_five_fourth)

# Testing: Inverse Distruntion for primes a^4-6b^4

In [ ]:
n_six_fourth = neg_inverse_distribution_primes_a_b_nth(-6,4)
n_six_fourth

In [ ]:
neg_testing(2000, 500, -6, 4, n_six_fourth) #rangeN, D, n, inverse_list

In [ ]:
neg_mod2_testing(2000,500,-6,4,n_six_fourth)

# Testing: Inverse Distruntion for primes a^4-11b^4

In [ ]:
n_eleven_fourth = neg_inverse_distribution_primes_a_b_nth(-11,4)
n_eleven_fourth

In [ ]:
neg_testing(2000, 500, -11, 4, n_eleven_fourth) #rangeN, D, n, inverse_list

In [ ]:
neg_mod2_testing(2000,500,-11,4,n_elevem_fourth)

# Testing: Inverse Distruntion for primes a^4 + 11b^4

In [ ]:
eleven_fourth = inverse_distribution_primes_a_b_nth(11, 4)
eleven_fourth

In [53]:
dist_eleven_fourth = distribution_primes_a_b(100,11,4, eleven_fourth) # def distribution_primes_a_b(N, D, n, LIST):
dist_eleven_fourth

[7, 6, 11, 8, 11, 12, 10]

In [ ]:
x = testing(1000, 11, 4, eleven_fourth)
x

In [ ]:
mod2_testing(2000,500,11, 4, eleven_fourth)

# Testing: Inverse Distruntion for primes a^4 + b^4

In [ ]:
one_fourth = inverse_distribution_primes_a_b_nth(1, 4)
one_fourth


In [9]:
one_4 = testing(1000, 1, 4, one_fourth)
one_4

Distribution of primes of the form a^4 + 1b^4


{'N = 100  [38, 51, 46, 47, 52, 55, 46, 56, 45]': 1.4736842105263157,
 'N = 200  [163, 161, 168, 173, 161, 185, 178, 178, 175]': 1.1490683229813665,
 'N = 300  [359, 354, 340, 347, 354, 369, 354, 348, 363]': 1.0852941176470587,
 'N = 400  [610, 590, 565, 580, 577, 608, 619, 590, 596]': 1.095575221238938,
 'N = 500  [913, 910, 845, 869, 878, 913, 904, 872, 887]': 1.0804733727810651,
 'N = 600  [1287, 1266, 1195, 1221, 1253, 1272, 1272, 1228, 1247]': 1.0769874476987447,
 'N = 700  [1670, 1698, 1596, 1637, 1647, 1686, 1672, 1642, 1636]': 1.0639097744360901,
 'N = 800  [2132, 2158, 2056, 2122, 2071, 2103, 2139, 2069, 2133]': 1.0496108949416343,
 'N = 900  [2630, 2708, 2598, 2589, 2612, 2607, 2651, 2602, 2639]': 1.0459636925453843,
 'N = 1000  [3189, 3275, 3182, 3112, 3178, 3132, 3235, 3110, 3210]': 1.0530546623794212}

In [ ]:
mod2_testing(2000,500,1, 4, one_fourth)

# Testing: Inverse Distruntion for primes a^4 + 2b^4

In [ ]:
two_fourth = inverse_distribution_primes_a_b_nth(2, 4)
two_fourth


In [54]:
two = testing(1000, 2, 4, two_fourth)
two

Distribution of primes of the form a^4 + 2b^4


{'N = 100  [17, 18, 25, 18, 17, 22, 17, 26]': 1.5294117647058822,
 'N = 200  [70, 66, 73, 65, 69, 68, 58, 76]': 1.3103448275862069,
 'N = 300  [140, 136, 143, 145, 153, 133, 127, 138]': 1.204724409448819,
 'N = 400  [224, 228, 241, 251, 241, 229, 215, 234]': 1.1674418604651162,
 'N = 500  [331, 336, 363, 376, 354, 352, 342, 361]': 1.13595166163142,
 'N = 600  [459, 468, 505, 521, 494, 482, 494, 497]': 1.1350762527233116,
 'N = 700  [621, 631, 652, 671, 669, 637, 665, 666]': 1.0805152979066022,
 'N = 800  [802, 825, 818, 840, 871, 796, 830, 861]': 1.0942211055276383,
 'N = 900  [1016, 1032, 1032, 1043, 1071, 981, 1027, 1063]': 1.091743119266055,
 'N = 1000  [1227, 1256, 1276, 1265, 1290, 1195, 1264, 1280]': 1.0794979079497908}

In [ ]:
mod2_testing(2000,500,2, 4, two_fourth)

# Testing: Inverse Distruntion for primes a^4 + 3b^4

In [ ]:
three_fourth = inverse_distribution_primes_a_b_nth(3, 4)
three_fourth

In [80]:
three = testing(1000, 3, 4, three_fourth)
three

Distribution of primes of the form a^4 + 3b^4


{'N = 100  [19, 27, 20, 20, 24, 18, 21, 28]': 1.5555555555555556,
 'N = 200  [67, 70, 80, 72, 74, 70, 78, 82]': 1.2238805970149254,
 'N = 300  [152, 149, 166, 149, 151, 155, 154, 152]': 1.1140939597315436,
 'N = 400  [250, 242, 272, 262, 238, 250, 254, 251]': 1.1428571428571428,
 'N = 500  [372, 376, 393, 392, 355, 373, 382, 375]': 1.1070422535211268,
 'N = 600  [510, 526, 542, 515, 501, 514, 532, 527]': 1.0818363273453093,
 'N = 700  [688, 688, 711, 672, 679, 669, 720, 682]': 1.0762331838565022,
 'N = 800  [868, 871, 904, 885, 869, 868, 895, 879]': 1.0414746543778801,
 'N = 900  [1063, 1061, 1117, 1085, 1078, 1079, 1117, 1085]': 1.052780395852969,
 'N = 1000  [1292, 1301, 1334, 1328, 1322, 1275, 1349, 1290]': 1.0580392156862746}

In [ ]:
mod2_testing(2000,500,3, 4, three_fourth)

# Testing: Inverse Distruntion for primes a^4 + 5b^4

In [ ]:
five_fourth = inverse_distribution_primes_a_b_nth(5, 4)
five_fourth

In [82]:
five_4 = testing(1000, 5, 4, five_fourth)
five_4

Distribution of primes of the form a^4 + 5b^4


{'N = 100  [5, 12, 7, 10, 14, 12, 10, 10]': 2.8,
 'N = 200  [20, 35, 34, 36, 40, 32, 42, 35]': 2.1,
 'N = 300  [57, 75, 75, 74, 71, 75, 80, 64]': 1.4035087719298245,
 'N = 400  [103, 124, 134, 127, 125, 116, 136, 122]': 1.3203883495145632,
 'N = 500  [157, 181, 197, 185, 189, 184, 201, 178]': 1.2802547770700636,
 'N = 600  [228, 247, 264, 258, 261, 243, 254, 255]': 1.1578947368421053,
 'N = 700  [309, 327, 350, 324, 348, 322, 332, 341]': 1.132686084142395,
 'N = 800  [396, 414, 437, 415, 438, 414, 429, 429]': 1.106060606060606,
 'N = 900  [502, 520, 537, 508, 517, 522, 525, 530]': 1.0697211155378485,
 'N = 1000  [621, 623, 648, 623, 605, 638, 634, 627]': 1.0710743801652893}

In [ ]:
mod2_testing(2000,500,5, 4, five_fourth)

# Testing: Inverse Distruntion for primes a^4 + 6b^4

In [ ]:
six_fourth = inverse_distribution_primes_a_b_nth(6, 4)
six_fourth

In [84]:
six_4 = testing(1000, 6, 4, six_fourth)
six_4

Distribution of primes of the form a^4 + 6b^4


{'N = 100  [9, 17, 10, 15, 16, 12, 14]': 1.8888888888888888,
 'N = 200  [50, 53, 45, 61, 45, 50, 45]': 1.3555555555555556,
 'N = 300  [94, 98, 98, 107, 83, 97, 100]': 1.2891566265060241,
 'N = 400  [152, 168, 163, 179, 150, 160, 158]': 1.1933333333333334,
 'N = 500  [229, 260, 238, 249, 230, 250, 227]': 1.145374449339207,
 'N = 600  [334, 345, 331, 336, 334, 346, 329]': 1.0516717325227964,
 'N = 700  [444, 447, 469, 436, 448, 456, 450]': 1.0756880733944953,
 'N = 800  [571, 557, 580, 548, 563, 593, 561]': 1.082116788321168,
 'N = 900  [732, 692, 699, 711, 692, 736, 687]': 1.0713245997088792,
 'N = 1000  [869, 841, 844, 853, 842, 886, 846]': 1.0535077288941737}

In [ ]:
mod2_testing(2000,500,6, 4, six_fourth)

# Testing: Inverse Distruntion for primes a^3 + 2b^3

In [ ]:
two_third = inverse_distribution_primes_a_b_nth(2, 3)
two_third

In [86]:
two_3 = testing(500, 2, 3, two_third)
two_3

Distribution of primes of the form a^3 + 2b^3


{'N = 100  [25, 31, 25, 27, 29, 25, 31, 35]': 1.4,
 'N = 200  [90, 107, 86, 101, 98, 88, 93, 95]': 1.244186046511628,
 'N = 300  [183, 200, 191, 185, 187, 175, 194, 188]': 1.1428571428571428,
 'N = 400  [311, 343, 327, 304, 320, 310, 309, 317]': 1.1282894736842106,
 'N = 500  [455, 505, 493, 486, 471, 490, 469, 470]': 1.10989010989011}

In [47]:
mod2_testing(2000,500,2,3, two_third)

Distribution of primes of the form a^3 + 2b^3
N = 500   [232, 254, 247, 235, 245, 235, 234, 239, 223, 251, 246, 251, 226, 255, 235, 231] 1.1434977578475336
N = 1000   [834, 858, 839, 864, 843, 840, 830, 856, 809, 864, 868, 844, 859, 878, 829, 852] 1.0852904820766378
N = 1500   [1827, 1800, 1721, 1765, 1791, 1782, 1761, 1785, 1745, 1793, 1785, 1785, 1795, 1830, 1787, 1799] 1.063335270191749
N = 2000   [3067, 2998, 2956, 3037, 3071, 2988, 3015, 3061, 2954, 3048, 3018, 3041, 3030, 3015, 3100, 3043] 1.0494245091401488
N = 2500   [4593, 4535, 4504, 4558, 4616, 4555, 4582, 4613, 4480, 4573, 4568, 4543, 4593, 4548, 4652, 4556] 1.0383928571428571


KeyboardInterrupt: 

# Testing: Inverse Distruntion for primes a^3 + 3b^3

In [ ]:
three_third = inverse_distribution_primes_a_b_nth(3, 3)
three_third

In [88]:
three_3 = testing(500, 3, 3, three_third)
three_3

Distribution of primes of the form a^3 + 3b^3


{'N = 100  [24, 30, 24, 28, 23, 27, 28]': 1.3043478260869565,
 'N = 200  [81, 87, 83, 69, 87, 91, 81]': 1.318840579710145,
 'N = 300  [163, 186, 169, 151, 188, 180, 168]': 1.2450331125827814,
 'N = 400  [276, 299, 291, 263, 307, 297, 290]': 1.167300380228137,
 'N = 500  [420, 448, 420, 394, 450, 452, 432]': 1.1472081218274113}

In [49]:
mod2_testing(2500,500,3,3, three_third)

Distribution of primes of the form a^3 + 3b^3
N = 500   [221, 228, 214, 204, 243, 220, 219, 199, 220, 206, 190, 207, 232, 213] 1.2789473684210526
N = 1000   [750, 750, 764, 733, 778, 750, 779, 747, 737, 761, 749, 759, 760, 751] 1.0627557980900408
N = 1500   [1563, 1583, 1627, 1582, 1635, 1563, 1582, 1615, 1580, 1569, 1600, 1591, 1617, 1596] 1.0460652591170825
N = 2000   [2689, 2676, 2745, 2685, 2740, 2659, 2705, 2725, 2691, 2684, 2704, 2687, 2712, 2727] 1.0323429860849944
N = 2500   [4102, 4038, 4147, 4074, 4108, 4014, 4099, 4085, 4071, 4033, 4078, 4082, 4052, 4107] 1.0331340308918784


# Testing: Inverse Distruntion for primes a^3 + 5b^3

In [ ]:
five_third = inverse_distribution_primes_a_b_nth(5, 3)
five_third

In [90]:
five_3 = testing(1000, 5, 3, five_third)
five_3

Distribution of primes of the form a^3 + 5b^3


{'N = 100  [13, 19, 14, 22, 15, 17, 15]': 1.6923076923076923,
 'N = 200  [48, 57, 56, 55, 56, 53, 51]': 1.1875,
 'N = 300  [110, 114, 118, 114, 115, 114, 111]': 1.0727272727272728,
 'N = 400  [181, 189, 179, 187, 198, 191, 193]': 1.106145251396648,
 'N = 500  [271, 283, 279, 274, 290, 280, 287]': 1.070110701107011,
 'N = 600  [385, 384, 388, 393, 391, 369, 403]': 1.0921409214092141,
 'N = 700  [508, 504, 502, 530, 516, 495, 526]': 1.0707070707070707,
 'N = 800  [639, 651, 648, 666, 660, 644, 651]': 1.0422535211267605,
 'N = 900  [798, 815, 811, 812, 809, 794, 808]': 1.026448362720403,
 'N = 1000  [965, 979, 978, 973, 975, 965, 970]': 1.0145077720207254}

In [ ]:
mod2_testing(2000,500,5,3, five_third)

# Testing: Inverse Distruntion for primes a^3 + 6b^3

In [ ]:
six_third = inverse_distribution_primes_a_b_nth(6, 3)
six_third

In [92]:
six_3 = testing(1000, 6, 3, six_third)
six_3

Distribution of primes of the form a^3 + 6b^3


{'N = 100  [9, 14, 9, 8, 15, 11]': 1.875,
 'N = 200  [34, 35, 36, 42, 42, 44]': 1.2941176470588236,
 'N = 300  [71, 75, 74, 83, 83, 87]': 1.2253521126760563,
 'N = 400  [124, 118, 132, 138, 141, 143]': 1.2118644067796611,
 'N = 500  [196, 181, 213, 194, 199, 224]': 1.2375690607734806,
 'N = 600  [278, 255, 293, 277, 254, 297]': 1.169291338582677,
 'N = 700  [360, 341, 388, 370, 352, 372]': 1.1378299120234605,
 'N = 800  [470, 434, 497, 473, 446, 477]': 1.1451612903225807,
 'N = 900  [576, 554, 597, 577, 555, 578]': 1.0776173285198556,
 'N = 1000  [691, 670, 719, 703, 681, 700]': 1.0731343283582089}

In [ ]:
mod2_testing(2000,500,6,3, six_third)

In [19]:
import numpy as np

In [23]:
hun_array = np.array([3273, 3297, 3279, 3306, 3241, 3256, 3330, 3216])

In [24]:
error_ = np.std(hun_array)/np.mean(hun_array)

In [25]:
error_

0.010516052610089251

In [22]:
np.std(np.array([254, 258, 258, 260, 235, 249, 254, 255]))/np.mean(np.array([254, 258, 258, 260, 235, 249, 254, 255]))

0.02948954767824944

In [26]:
import math

In [28]:
math.dev([3273, 3297, 3279, 3306, 3241, 3256, 3330, 3216])

AttributeError: module 'math' has no attribute 'dev'